In [66]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [81]:
data = pd.read_csv('pred_data/ready_pred.csv')

In [82]:
filename = '../saved_models/faint/rfc_RScv_faint.sav'
model = pickle.load(open(filename,'rb'))

In [83]:
X = data[['RA','DEC','u - g', 'u - r', 'u - i', 'u - z', 'u - j', 'u - k',
       'u - ch1', 'u - ch2', 'g - r', 'g - i', 'g - z', 'g - k', 'g - j',
       'g - ch1', 'g - ch2', 'r - i', 'r - z', 'r - j', 'r - ch1', 'r - k',
       'r - ch2', 'i - z', 'i - j', 'i - k', 'i - ch1', 'i - ch2', 'z - j',
       'z - k', 'z - ch1', 'z - ch2', 'j - ch1', 'j - ch2', 'j - k', 'k - ch1',
       'k - ch2', 'ch1 - ch2']]

In [84]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

clean_dataset(X)
np.any(np.isnan(X))

/var/folders/dq/sc1tlll95vbbtbqb_dbg21km0000gn/T/ipykernel_6972/4193781425.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


False

In [85]:
len(X)

347

In [86]:
x_m = X.merge(data[['class_cleaned','data']],left_index=True,right_index = True)

In [87]:
x_m.columns

Index(['RA', 'DEC', 'u - g', 'u - r', 'u - i', 'u - z', 'u - j', 'u - k',
       'u - ch1', 'u - ch2', 'g - r', 'g - i', 'g - z', 'g - k', 'g - j',
       'g - ch1', 'g - ch2', 'r - i', 'r - z', 'r - j', 'r - ch1', 'r - k',
       'r - ch2', 'i - z', 'i - j', 'i - k', 'i - ch1', 'i - ch2', 'z - j',
       'z - k', 'z - ch1', 'z - ch2', 'j - ch1', 'j - ch2', 'j - k', 'k - ch1',
       'k - ch2', 'ch1 - ch2', 'class_cleaned', 'data'],
      dtype='object')

In [88]:
x_m.reset_index(inplace = True)
x_input =  x_m[['u - g', 'u - r', 'u - i', 'u - z', 'u - j', 'u - k',
       'u - ch1', 'u - ch2', 'g - r', 'g - i', 'g - z', 'g - k', 'g - j',
       'g - ch1', 'g - ch2', 'r - i', 'r - z', 'r - j', 'r - ch1', 'r - k',
       'r - ch2', 'i - z', 'i - j', 'i - k', 'i - ch1', 'i - ch2', 'z - j',
       'z - k', 'z - ch1', 'z - ch2', 'j - ch1', 'j - ch2', 'j - k', 'k - ch1',
       'k - ch2', 'ch1 - ch2']]

In [89]:
threshold = 0.5165

predicted_proba = model.predict_proba(x_input)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

In [90]:
result = pd.DataFrame(columns = ['RA','DEC','P of a non-quasar', 'P of a quasar','Prediction', "? or no_obs",'data'])

In [91]:
result['P of a non-quasar'] = predicted_proba[:,0]
result['P of a quasar'] = predicted_proba[:,1]
result['Prediction'] = predicted
result['RA'] = x_m['RA']
result['DEC'] = x_m['DEC']
result['? or no_obs'] = x_m['class_cleaned']
result['data'] = x_m['data']

In [92]:
result

,RA,DEC,P of a non-quasar,P of a quasar,Prediction,? or no_obs,data
0,0.062537,-0.578794,0.536828,0.463172,0,none,Teddy
1,0.069474,0.089717,0.609944,0.390056,0,?,Teddy
2,0.134173,0.695095,0.496965,0.503035,0,none,Teddy
3,0.148627,0.954599,0.505336,0.494664,0,none,Teddy
4,0.192235,-0.534088,0.759080,0.240920,0,none,Teddy
...,...,...,...,...,...,...,...
342,231.558273,-1.886450,0.180126,0.819874,1,no_obs,Glikman
343,232.727631,0.980603,0.037304,0.962696,1,no_obs,Glikman
344,234.152176,6.422978,0.223686,0.776314,1,?,Glikman
345,234.151627,8.965731,0.248093,0.751907,1,no_obs,Glikman


In [93]:
result.to_csv('result/result_v1.csv')

In [94]:
count_class = pd.DataFrame(result['Prediction'].value_counts()).reset_index()
count_class.rename(columns = {'index' : 'class_cleaned','class_cleaned':'occ'},inplace = True)
count_class

,class_cleaned,Prediction
0,1,186
1,0,161
